# Python and Google Earth Engine


# To do! 
- [ ] Add learning goals!! 
- [ ] add something that sais run this in jpuyter! 
- [ ] add final analysis part
- [ ] create exercise




Many of you will know or have scrolled around in [google earth](https://earth.google.com/web/). Within Google Earth it is possible to expplore the earth behind your computer. Through the application a lot of different maps and information can be found, some straight forward such as the normal google maps base map, but also layers such as the age of the sea floor. Additionally, (or primarily if you ask geospatial people) it is a powerful cloud based computational platform where geospatial analysis can be carried out. All this information and all tools are gathered, and made available to everybody (with a google account...) in the "engine" behind Google Earth: Google Earth Engine (GEE). GEE was originally developed and made available as a _JavaScript API_. What does that exacly mean? Let's break it up into Javascript and API. 

### API
An API is a "Application Programming Interface", which is a standard language (or a set of rules) with wich pieces of software can commincate with eachother. API's are used for for example [accessing data](https://openweathermap.org/api/one-call-3#how) or to interact with the [command line tool: GDAL](https://geoscripting-wur.github.io/PythonRaster/) like we did yesterday. 


### JavaScript  
JavaScript (JS) is a widely used programming language, primarily for interacting with website. Javascript is an object oriented programming language, which means that it focuses on creating reusable code through objects. Objects can encapsulate both data (properties) and functionality (methods) in one single object. Object Oriented Programming (OOP) is a programing paradigm which is slightly different than how we have programmed so far. Python is also, or can also be used as, an OOP language. Working with the GEE Python API is a great oportunity to get familiair with object oriented programming while at the same time work with the extremely powerful engine from google. 

# Object Oriented Programming in Python
In this tutorial we will learn about how to interact with GEE. Before we do this we need to be able to think in a Object Oriented Programming way. Therefor we will need to understand how OOP is implemented in Python. 

In Python, objects are represented using classes. A class is a blueprint that defines the structure and behavior of objects. It encapsulates data (properties) and functions (methods) into a single object.

To create a class, we use the `class` keyword, followed by the class name. Here's an example of a simple class called `Person`:

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name  # < this is a property
        self.age = age    # < this is also a property
    
    def greet(self):  # < This "function" is a method
        print(f"Hello, my name is {self.name} and I'm {self.age} years old.")

In the above code, the __init__ method is a special method called a *constructor*.  It is invoked automatically when an object is created from the class. The `self` parameter refers to the instance of the class and allows access to its properties and methods. In this example it means that every time self is passed to the class or a method in the class, this has access to all its own properties and methods. Generally, always when a method is created we give the self key word as one of the parameters. 

The Person class has two properties (name and age) and one method (greet). The greet method prints a greeting message with the person's name and age.

To create an instance of the Person class, you simply call the class as if it were a function and assign it to a variable:

In [ ]:
person1 = Person("Alice", 25)
person2 = Person("Bob", 30)

We created 2 _objects_, which are _instances_ of the _class_ _Person_, called `person1` and `person2`. We can now access the properties and call the methods of these objects:

```python
print(person1.name)  # Output: Alice
print(person2.age)   # Output: 30
person1.greet()      # Output: Hello, my name is Alice and I'm 25 years old.
person2.greet()      # Output: Hello, my name is Bob and I'm 30 years old.
```

This is a simple example, but you can imagine that these can get very complex. For example take a look at how a GeoPandas GeoSeries is created [here](https://github.com/geopandas/geopandas/blob/main/geopandas/geoseries.py). It is very complicated and hard to read, but very well documented so try to understand some of its functionality! The method [`to_json`](https://github.com/geopandas/geopandas/blob/main/geopandas/geoseries.py#L1209) contains very straight forward functionality (but it might still be hard to read).

One of the things that might occur to you is that the classes are defined slightly different than we have learned. In the geopandas example we see that a class is created using the following code: 

```python 
class GeoSeries(GeoPandasBase, Series):
    
```

While we learned we just had to call the following coode:

```python 
class GeoSeries:
```

The difference is that the code in the brackets is how "inheritance" is implemented. The class will _inherit_ all the functionality of the classes passed as arguments, in this case `GeoPandasBase` and `Series`. The `Series` object refers to the [Pandas.Series](https://github.com/pandas-dev/pandas/blob/main/pandas/core/series.py#L243C15-L243C15) object containing 6000 lines of code containing all sorts of functionality. In this way, the GeoPandas series contains all the functionality that is implemented in pandas *plus* all the functionality from GeoPandasBase *plus* all the functionality implemented in GeoPandas.Series itself.

Pfew, that's a lot of very complicated code and maybe a bit very overwelming and you do not need to understand all of it. The important thing to understand is the value of classes, objects and inheritance. How does this work in a simple example? 

So we established that with inheritance we can allow classes to take all the functionality from other classes and build on that. So let's create a new object called `Student`, which will inherit all properties and methods from `Person` which we created earlier. 


In [ ]:
class Student(Person):
    def __init__(self, name, age, student_id):
        super().__init__(name, age)
        self.student_id = student_id
    
    def study(self):
        print(f"{self.name} is studying.")



In the above code, the `Student` class inherits from the `Person` class (this class we will from now refer to as the superclass). It extends the functionality by adding a new property (student_id) and a new method (study). The `super()` function is used to call the superclass's __init__ method, allowing the subclass to initialize the inherited properties. 

The student class now contains methods and properties from Person as well as from Student: 

```python
student = Student("Eve", 22, "123456")
print(student.name)         # Output: Eve
print(student.student_id)   # Output: 123456
student.greet()             # Output: Hello, my name is Eve and I'm 22 years old.
student.study()             # Output: Eve is studying.
```

# Google Earth Engine

As already introduced, Google Earth Engine is a large collection of geospatial data with poerwful compitational capabilities. It allows users to analyze and visualize geospatial data on a global scale, making it a valuable tool for research about for example environmental monitoring and resources managent. 

One of the key features of GEE is its collection of raster datasources. Have a look at the [catalogue](https://developers.google.com/earth-engine/datasets/catalog) to see what is out there. The collection also contains raster time series and non-raster data although. For some use cases, check out [this article by Pérez-Cutillas and colleagues](https://www.sciencedirect.com/science/article/pii/S2352938522002154) from this year. They did a systematic review of research done using GEE. From their article:  

> The results of the meta-analysis following the systematic review showed that: (i) the Landsat 8 was the most widely-used satellite (25%); (i) the non-parametric classification methods, mainly Random Forest, were the most recurrent algorithms (31%); and (iii) the water resources assessment and prediction were the most common methodological applications (22%).
> 
> -- Pérez-Cutillas et al., 2023

## Night time light emision 
In this tutorial we will do a relatively straight forward analysis to show you some often used tools, but more importantly we want to show you how to work with GEE in python, how to find tools and documentation and how to read the (JS) documentation.

The analysis we will carry out is a comparison of how much light is emited in the Netherlands during the night through time. We will run you step by step through the procedure. The data we will use is a worldwide dataset from 2012 to now so feel free to carry out the analysis on a different area. 

The environment we will use will need the following dependencies. Save the following yaml in a new file called env.yaml and create and activate an environment: 

```yaml
name: pythonGee
channels:
  - conda-forge
dependencies:
  - python=3.10
  - jupyter
  - earthengine-api
  - geemap
  - pygadm
  - geopandas
  - geojson
```

To test try and import the following packages: 


In [1]:
import ee
import geemap 
import pygadm
import geopandas as gpd
import geojson

Google Earth engine is a google application, so not surprisingly you will need a google acount to be authorized using this account for working with their tools. The authentication will be done by a procedure started by calling the `Authenticate` function from `ee`. Follow these steps: 
1. Follow the url that will print after running the Authenticate function
1. require you to create a token by loging into your google account. 
1. Click Continue when you get the warning that google has not verified this app. This means that Google did not verify the code we are about to run, and it prevents you from your account being used by others. 
1. Next select all the tick boxes, this will allow you to use GEE data and it allows you to manage your data and permissions in the cloud storage, where the actual analysis will be run.
1. lastly you wil need to copy and pased the authorization code (and press enter) in the textbox which will apear after running the Authenticate method. 

In [ ]:
ee.Authenticate()

You now have (hopefully) succesfully authenticated. Next we have to initialize using the `Initialize()` method: 

In [2]:
ee.Initialize()

If you get no further warnings or errors you have succesfullt authenticated and are ready to work with the earth engine library in python! Congrats! Next time you are working in the same environment, you will not have to authenticate again. You can just run the `Initialize` function and off you go! 

## Night time data

So we want to analyze how the light emitance has developed over the last 10 years. For this we will compare per province how much light was emited per month. 

For the analysis we will need data about emitted light. We will use the VIIRS Nighttime data. Read more about the data [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG). It is monthly average radiance composite images using nighttime data from the Visible Infrared Imaging Radiometer Suite (VIIRS) Day/Night Band (DNB). You can have a look at the data in [this esri viewer](https://www.arcgis.com/apps/mapviewer/index.html?layers=edabcbb5407547f5bc883018eb6e7986). To use this data we can create a `ImageCollection` object. Have a look at the [documentation](https://developers.google.com/earth-engine/apidocs/ee-imagecollection). We can see that for the implementation example we have 2 options. 1 is the JavaScript option and one is the Python Google Colab option. This is because Google is implementing all the JavaScript functionality also in python. Well actually, as far as we know all functionality seems to be implemented, however not all the documentation or example has been created for python. Luckily, if you switch between the JS and the Python documentation, we will see not much differences. Because we do not know at what point what python documentation will be available and because it is a good exercise to be able to read JavaScript documentation we will refer to the JavaScript documentation and point out important differences, but feel free to use the Python documentation when available. 

An image collection can be created using the following command:

In [3]:
# Import the night time light emision data.
viirs_image_collection = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")

You can see that this is the exact same code as in the JS documentation. However, they limit the collection to 3 images only, whereas we imported all images. After all we want to study the development over the entire timespan. An important difference between the JS API and the Python API is that to get information about an object we have to call the `getInfo()` method. So instead of in JS printing information as follows: 
```JavaScript
print('Image collection from a string', ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG"));
```

We can use the following command:

In [ ]:
print(type(ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").getInfo()))

We need to call the `getInfo` method to convert the object information to a dictionary where we could extract useful information from. We can also show the ImageCollection object in jupyter by calling the object itself. 

In [ ]:
ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")

We can see that the object has some properties. it has a `type`, `id`, `version`, an empty list as `bands`, 23 `properties` and 132 `features`. Note that these are very similair to what you would expect a [geojson](https://geojson.org/) containing a FeatureCollection to look like. 

The type, id and version are information from google about the data and the object type. At this point the object has an empty list as bands, which means at this points no bands. It has 23 properties. Note that these properties are properties from the collection, not from the individual images, so they contain information such as that it is a montly collection. Lastly the collection contains features: a list of 132 elements. Each element is an Image with again a `type`, `id`, `version`, `bands` and `properties`, but in this case no features. It is not a collection in the end. 

These features are the individual images, 1 per timestamp, so 11 years of 1 image per month. Let's look at one of these images. 

## GEE as a viewer! 

So as said before, Google Earth Engine is the engine behind Google Earth. It not only is a data collection, it also contains tools and a viewer. To interface with this viewer we will use another package called `geemap`. [Geemap](https://geemap.org/) is not only a viewer, it also contains very handy tools, for example to convert data to and from GEE native objects. More on that later.

To create amap we have to follow a view steps, which we will gracefully take from the [geemap getting started page](https://geemap.org/get-started/#add-earth-engine-data) and modify it a little for our use case.  

We will take the following steps: 
1. Initialize a map, with a starting position and a zoom leve. This map will stay where it is, and we will add layers to it as we go.
1. We will select 1 image from our image collection
1. We will add this image to the map 
1. We will also add a boundary of the netherlands to the same map. 

In [4]:
# set our initial map parameters for Wageningen
center_lon = 51.962589
center_lat = 5.669627
zoomlevel = 8


# The following line will initialize the map
Map = geemap.Map(center=[center_lon, center_lat], zoom=zoomlevel, layer_ctrl=True)

# The following line will actually call and show the map:
Map


Map(center=[51.962589, 5.669627], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

The map will stay and live in this window where it is. It is an interactive map with at this moment 1 standard basemap. To start over again you can call `Map.clear()`, that will remove all layers and other objects from the map and you will have to initialize it again. Step 1 done! Next step is to select one image from the image collection and to add it to the map. 

In [5]:
# Initial date of interest (inclusive).
from_date = '2017-01-01'
# Final date of interest (exclusive).
to_date = '2020-01-01'

image = viirs_image_collection.filterDate(from_date, to_date).sort("system:time_start", True).first()

visualization_params = {
    'min': 0,
    'max': 10,
    'opacity': 0.5,
    'bands': ['avg_rad']
}

Map.addLayer(ee_object = image, 
             vis_params = visualization_params,
             name = 'My First Image')


In the map you can see that an image is added. But what did the code do? Try to understand it for yourself and look at the documentation if needed, but we will also explain here. 

In the first line a couple of things happened: 

```python 
image = viirs_image_collection.sort("system:time_start", True).first()
```

1. Firstly we sort the collection using a method of the imageCollection, see the [documentation here](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-sort). It sorts the collection on a property, in this case that is `system:time_start`. Each image in the colllection has this property, it specifies the start date of the image (the values are an average over a period of time, from this timestamp to the `system:time_end` property). Lastly the boolean True refers to the argument `ascending`. In this case we want to sort the collection on `system:time_start` and we want to sort it ascending. It returns an image collection again.
1. Secondly, we select only the images between 2 dates that we defined, from 2017 to 2020. 
1. Lastly we call the method `First`. This is straight forward, the collection is now sorted as described above, and we want to return the first element of this collection. 

Next we define some visualization parameters we will use to put the image on the map. We select the range of images, we use an opacity and we want to show 1 band: `avg_rad`. If we look at the image we can see it has 2 bands: `avg_rad` and `cf_cvg`. See what they mean [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG#bands).

We then add the image to the map using the `AddLayer` method. It needs an earth engine object, visualization paramaters we defined earlier and a name. The last two are defaulted to nothing. 

If we look at the map we can see the image has indeed appeared. Great! Second and third step done! 

So for the last step we also want to add the boundaries of the netherlands. For getting the data we will use another package called `pygadm`. Pygadm is a dataset project from the Berkely University, and it's a great way to get world wide data on administrative boundaries on multiple scales. 

In [7]:
netherlands = pygadm.get_items(admin='NLD', content_level=1)
ee_netherlands = geemap.geopandas_to_ee(netherlands)
Map.addLayer(ee_object = ee_netherlands, 
             name = 'Netherlands Boundaries')
Map.addLayerControl()

Using the `get_items` function, we can get the boundaries on content level 1, which are provinces in the Netherlands. We get this data in `GeoPandas GeoDataFrame` format, which we can not directly add to the map, however we can use a handy conversion tool coming straight from the `geemap` package. The earth engine geometry resulting from this we can put on the map quite easily. 

For more examples and tools, look at the [examples](https://github.com/gee-community/geemap/tree/master/examples/notebooks) supplied by geemap themselves! Especially note that one can [export a map to html](https://github.com/gee-community/geemap/blob/master/examples/notebooks/21_export_map_to_html_png.ipynb), that might come in useful during the project.